<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex07/larissa_santesso/Aula_7_Exerc%C3%ADcio_Larissa_Santesso_modelofinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = "Larissa Antonelli Santesso"
print(f'Meu nome é {nome}')

Meu nome é Larissa Antonelli Santesso


#  Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural simples para prever a proxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Língua".

Este dataset já possui um tamanho razoável e é bem provável que você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [1]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |████████████████████████████████| 6.6 MB 40.5 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Mon May 16 22:43:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check which GPU we are using (2nd run)
!nvidia-smi

Tue May 17 17:21:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


In [5]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.tokensIds_n = []
        self.y = []
        for text in texts:
            tokens_ids = tokenize(text, tokenizer)
            for i in range(len(tokens_ids)-context_size):
                self.tokensIds_n.append(tokens_ids[i:i+context_size])      
                self.y.append(tokens_ids[i+context_size])
                
    def __len__(self):  
        return len(self.tokensIds_n)

    def __getitem__(self, idx):
        
        return torch.tensor(self.tokensIds_n[idx]).long(), torch.tensor(self.y[idx]).long()

## Teste se sua implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


In [9]:
first_batch_input

tensor([[ 3396, 10303,   125],
        [ 1660,  5971,   785],
        [ 5971,   785,   125],
        [  785,   125,  1847],
        [  125,  1847, 13779]])

In [10]:
first_batch_target

tensor([13239,   125,  1847, 13779, 15616])

# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-18 21:55:44--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.128, 173.194.210.128, 173.194.216.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   136MB/s    in 0.9s    

2022-05-18 21:55:45 (136 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [35]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

#print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

In [13]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [9]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim, hidden_size):
        """
        Implements the Neural Language Model proposed by Bengio et al."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            hidden_size (int): Size of the hidden layer.
        """
        super(LanguageModel, self).__init__()

        self.context_size = context_size
        self.embeddings_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(self.context_size*self.embeddings_dim, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size*2) # adicionei uma camada a mais que o modelo original de Bengio
        self.linear3 = nn.Linear(hidden_size*2, vocab_size, bias=False)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
        """
        out =  self.embeddings(inputs).view(-1,self.context_size*self.embeddings_dim)
        out = self.linear1(out)
        out = self.relu1(out)
        out = self.linear2(out)
        out = self.relu2(out)
        out = self.linear3(out)
        
        return out

## Teste o modelo com um exemplo

In [36]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [37]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 19494912


## Assert da Perplexidade


In [17]:
def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity.
    """
    crossentropy =  nn.functional.cross_entropy(logits,target)
    p = torch.exp(crossentropy)
    #p = np.exp(crossentropy.item())

    return p


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=2000)
print('Passou o no assert da perplexidade')

my perplexity:              29788
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 200_000_000
eval_every_steps = 5000
lr = 3e-5
compare=1e10

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                
                if valid_ppl<compare:
                    compare=valid_ppl
                    torch.save(model, "gdrive/MyDrive/Colab Notebooks/"+"model03.pt")
                    with open("gdrive/MyDrive/Colab Notebooks/"+"valid_ppl_model03.txt", 'w') as f:
                        f.write("%s\n" % valid_ppl)
                    f.close()

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29601.35, valid ppl: 29523.83
5000 steps; 5120000 examples so far; train ppl: 1643.86, valid ppl: 1232.53
10000 steps; 10240000 examples so far; train ppl: 1032.56, valid ppl: 888.63
15000 steps; 15360000 examples so far; train ppl: 807.39, valid ppl: 742.06
20000 steps; 20480000 examples so far; train ppl: 694.06, valid ppl: 649.06
25000 steps; 25600000 examples so far; train ppl: 614.35, valid ppl: 577.70
30000 steps; 30720000 examples so far; train ppl: 546.86, valid ppl: 522.68
35000 steps; 35840000 examples so far; train ppl: 495.75, valid ppl: 478.36
40000 steps; 40960000 examples so far; train ppl: 453.89, valid ppl: 440.98
45000 steps; 46080000 examples so far; train ppl: 421.97, valid ppl: 409.35
50000 steps; 51200000 examples so far; train ppl: 392.51, valid ppl: 382.76
55000 steps; 56320000 examples so far; train ppl: 368.45, valid ppl: 361.60
60000 steps; 61440000 examples so far; train ppl: 344.28, valid ppl: 343.50
65000 steps; 66560

In [ ]:
with open("gdrive/MyDrive/Colab Notebooks/"+"valid_ppl_model03.txt") as f:
    txt = list(f)
    compare = float(txt[-1])
    f.close()

In [ ]:
max_examples = 600_000_000
eval_every_steps = 5000
lr = 3e-5
print(compare)

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)

model = torch.load("gdrive/MyDrive/Colab Notebooks/"+"model03.pt")
model.to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                
                if valid_ppl<compare:
                    compare=valid_ppl
                    torch.save(model, "gdrive/MyDrive/Colab Notebooks/"+"model03_run02.pt")
                    with open("gdrive/MyDrive/Colab Notebooks/"+"valid_ppl_model03_run02.txt", 'w') as f:
                        f.write("%s\n" % valid_ppl)
                    f.close()

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

194.7833877148957
0 steps; 0 examples so far; train ppl: 179.25, valid ppl: 194.82
5000 steps; 5120000 examples so far; train ppl: 183.87, valid ppl: 196.37
10000 steps; 10240000 examples so far; train ppl: 180.88, valid ppl: 193.46
15000 steps; 15360000 examples so far; train ppl: 179.46, valid ppl: 191.38
20000 steps; 20480000 examples so far; train ppl: 178.24, valid ppl: 188.96
25000 steps; 25600000 examples so far; train ppl: 176.59, valid ppl: 187.42
30000 steps; 30720000 examples so far; train ppl: 171.86, valid ppl: 186.13
35000 steps; 35840000 examples so far; train ppl: 170.14, valid ppl: 184.85
40000 steps; 40960000 examples so far; train ppl: 169.51, valid ppl: 183.67
45000 steps; 46080000 examples so far; train ppl: 168.65, valid ppl: 182.24
50000 steps; 51200000 examples so far; train ppl: 168.06, valid ppl: 180.98
55000 steps; 56320000 examples so far; train ppl: 166.44, valid ppl: 179.88
60000 steps; 61440000 examples so far; train ppl: 162.17, valid ppl: 178.88
65000 s

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [38]:
model = torch.load("gdrive/MyDrive/Colab Notebooks/"+"model03_run02.pt", map_location=device)
model.to(device)

LanguageModel(
  (embeddings): Embedding(29794, 128)
  (linear1): Linear(in_features=1152, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=29794, bias=False)
  (relu1): ReLU()
  (relu2): ReLU()
)

In [39]:
test_loader = DataLoader(test_dataset, batch_size=64)

def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 141.6671717478626


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [40]:
prompt = "Temos que pensar no futuro e guardar o que aprendemos na "  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    input_ids_truncated = torch.tensor(input_ids_truncated).long()
    logits = model(torch.LongTensor(input_ids_truncated).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Temos que pensar no futuro e guardar o que aprendemos na vida
Temos que pensar no futuro e guardar o que aprendemos na vida de
Temos que pensar no futuro e guardar o que aprendemos na vida de um
Temos que pensar no futuro e guardar o que aprendemos na vida de um homem
Temos que pensar no futuro e guardar o que aprendemos na vida de um homem,
Temos que pensar no futuro e guardar o que aprendemos na vida de um homem, e
Temos que pensar no futuro e guardar o que aprendemos na vida de um homem, e que
Temos que pensar no futuro e guardar o que aprendemos na vida de um homem, e que não
Temos que pensar no futuro e guardar o que aprendemos na vida de um homem, e que não se
Temos que pensar no futuro e guardar o que aprendemos na vida de um homem, e que não se pode


In [41]:
prompt = "Uma xícara de café e um livro são as"  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    input_ids_truncated = torch.tensor(input_ids_truncated).long()
    logits = model(torch.LongTensor(input_ids_truncated).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Uma xícara de café e um livro são as melhores
Uma xícara de café e um livro são as melhores opções
Uma xícara de café e um livro são as melhores opções de
Uma xícara de café e um livro são as melhores opções de vida
Uma xícara de café e um livro são as melhores opções de vida,
Uma xícara de café e um livro são as melhores opções de vida, e
Uma xícara de café e um livro são as melhores opções de vida, e a
Uma xícara de café e um livro são as melhores opções de vida, e a sua
Uma xícara de café e um livro são as melhores opções de vida, e a sua vida
Uma xícara de café e um livro são as melhores opções de vida, e a sua vida,


In [42]:
prompt = "Cachorro e gato são os melhores amigos do"  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    input_ids_truncated = torch.tensor(input_ids_truncated).long()
    logits = model(torch.LongTensor(input_ids_truncated).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Cachorro e gato são os melhores amigos do mundo
Cachorro e gato são os melhores amigos do mundo.
Cachorro e gato são os melhores amigos do mundo. A
Cachorro e gato são os melhores amigos do mundo. A partir
Cachorro e gato são os melhores amigos do mundo. A partir de
Cachorro e gato são os melhores amigos do mundo. A partir de agora
Cachorro e gato são os melhores amigos do mundo. A partir de agora,
Cachorro e gato são os melhores amigos do mundo. A partir de agora, a
Cachorro e gato são os melhores amigos do mundo. A partir de agora, a partir
Cachorro e gato são os melhores amigos do mundo. A partir de agora, a partir de


In [59]:
prompt = "A exposição da Tarsila ocorrerá na cidade"  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    input_ids_truncated = torch.tensor(input_ids_truncated).long()
    logits = model(torch.LongTensor(input_ids_truncated).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

A exposição da Tarsila ocorrerá na cidade de
A exposição da Tarsila ocorrerá na cidade de São
A exposição da Tarsila ocorrerá na cidade de São Paulo
A exposição da Tarsila ocorrerá na cidade de São Paulo.
A exposição da Tarsila ocorrerá na cidade de São Paulo. O
A exposição da Tarsila ocorrerá na cidade de São Paulo. O que
A exposição da Tarsila ocorrerá na cidade de São Paulo. O que é
A exposição da Tarsila ocorrerá na cidade de São Paulo. O que é o
A exposição da Tarsila ocorrerá na cidade de São Paulo. O que é o primeiro
A exposição da Tarsila ocorrerá na cidade de São Paulo. O que é o primeiro passo


In [56]:
prompt = "As mulheres estão cada vez mais presentes comparado a"  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    input_ids_truncated = torch.tensor(input_ids_truncated).long()
    logits = model(torch.LongTensor(input_ids_truncated).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

As mulheres estão cada vez mais presentes comparado a um
As mulheres estão cada vez mais presentes comparado a um ano
As mulheres estão cada vez mais presentes comparado a um ano de
As mulheres estão cada vez mais presentes comparado a um ano de 2013
As mulheres estão cada vez mais presentes comparado a um ano de 2013.
As mulheres estão cada vez mais presentes comparado a um ano de 2013. O
As mulheres estão cada vez mais presentes comparado a um ano de 2013. O primeiro
As mulheres estão cada vez mais presentes comparado a um ano de 2013. O primeiro passo
As mulheres estão cada vez mais presentes comparado a um ano de 2013. O primeiro passo é
As mulheres estão cada vez mais presentes comparado a um ano de 2013. O primeiro passo é que
